Función para obtener la cantidad de items y porcentaje de contenido gratuito por año

# Primera funcion

In [19]:
import pyarrow.parquet as pq
import pandas as pd

def DeveloperFreeContent(desarrollador: str):
    # Cargar los archivos Parquet y seleccionar las columnas necesarias
    steam_games_df = pq.read_table("dataset/steam_games.parquet", columns=['item_id', 'developer', 'release_year', 'price']).to_pandas()

    # Paso 1: Filtrar las filas que coinciden con el desarrollador
    developer_df = steam_games_df[steam_games_df['developer'] == desarrollador]

    if developer_df.empty:
        return {"error": "Desarrollador no encontrado"}

    # Paso 2: Utilizar agregaciones de Pandas para calcular las estadísticas
    result = (developer_df.groupby('release_year')
             .agg({'item_id': 'count', 'price': (lambda x: f"{(x == 0).sum() / len(x) * 100:.2f}%" if len(x) > 0 else "0%")})
             .reset_index()
             .rename(columns={'release_year': 'Año', 'item_id': 'Cantidad de Items', 'price': 'Contenido Free'})
             .to_dict(orient='records'))

    return result

In [20]:
f1 = DeveloperFreeContent('Valve')
f1

[{'Año': 1998, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 1999, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2000, 'Cantidad de Items': 2, 'Contenido Free': '0.00%'},
 {'Año': 2001, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2003, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2004, 'Cantidad de Items': 5, 'Contenido Free': '0.00%'},
 {'Año': 2005, 'Cantidad de Items': 1, 'Contenido Free': '100.00%'},
 {'Año': 2006, 'Cantidad de Items': 2, 'Contenido Free': '0.00%'},
 {'Año': 2007, 'Cantidad de Items': 3, 'Contenido Free': '33.33%'},
 {'Año': 2008, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2009, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2010, 'Cantidad de Items': 2, 'Contenido Free': '50.00%'},
 {'Año': 2011, 'Cantidad de Items': 1, 'Contenido Free': '0.00%'},
 {'Año': 2012, 'Cantidad de Items': 2, 'Contenido Free': '50.00%'},
 {'Año': 2014, 'Cantidad de Items': 7, 'Contenido Free': 

In [3]:
f1_1 = f1[0]['Año']
f1_1

1998

# Segunda funcion

## Función para obtener la cantidad de dinero gastado, %de recomendacion y numero de items del usuario

In [17]:
import pyarrow.parquet as pq
import pandas as pd

def UserData(User_id: str):
    # Cargar los archivos Parquet y seleccionar las columnas necesarias
    user_items_df = pq.read_table("dataset/user_items.parquet", columns=['user_id', 'items_count', 'steam_id']).to_pandas()
    user_items_list_df = pq.read_table("dataset/user_items_list.parquet", columns=['steam_id', 'item_id']).to_pandas()
    user_reviews_df = pq.read_table("dataset/user_reviews.parquet", columns=['steam_id', 'recommend']).to_pandas()
    steam_games_df = pq.read_table("dataset/steam_games.parquet", columns=['item_id', 'price']).to_pandas()

    # Filtrar la fila correspondiente al user_id en user_items_df
    user_row = user_items_df[user_items_df['user_id'] == User_id]

    if user_row.empty:
        return {"error": "Usuario no encontrado"}

    # Obtener la cantidad de items del usuario
    items_count = user_row['items_count'].iloc[0]

    # Obtener el steam_id del usuario
    steam_id = user_row['steam_id'].iloc[0]

    # Filtrar las filas correspondientes a este steam_id en user_items_list_df
    user_item_rows = user_items_list_df[user_items_list_df['steam_id'] == steam_id]

    # Obtener los item_ids correspondientes
    item_ids = user_item_rows['item_id']

    # Filtrar las filas correspondientes en steam_games_df y obtener los precios
    prices = steam_games_df[steam_games_df['item_id'].isin(item_ids)]['price']

    # Calcular el dinero gastado por el usuario
    money_spent = prices.sum()

    # Filtrar las reviews correspondientes a este steam_id
    user_reviews = user_reviews_df[user_reviews_df['steam_id'] == steam_id]

    # Realizar agregaciones para calcular el porcentaje de recomendación
    total_reviews = len(user_reviews)
    recommended_reviews = user_reviews[user_reviews['recommend'] == 1]
    recommendation_percentage = (len(recommended_reviews) / total_reviews) * 100 if total_reviews > 0 else 0

    # Crear el resultado final
    result = {
        "Usuario X": User_id,
        "Dinero gastado": f"{money_spent:.2f} USD",
        "% de recomendación": f"{recommendation_percentage:.2f}%",
        "Cantidad de items": int(items_count)
    }

    return result

In [18]:
f2 = UserData('evcentric')
f2

{'Usuario X': 'evcentric',
 'Dinero gastado': '1584.90 USD',
 '% de recomendación': '0.00%',
 'Cantidad de items': 137}

# Tercera funcion

## Función para obtener el usuario con mas cantidad de horas jugadas por año de un genero

In [1]:
import pyarrow.parquet as pq
import pandas as pd


def UserForGenre(genero: str):

    # Cargar los archivos Parquet una vez al inicio de la aplicación
    steam_games_genre_df = pq.read_table("dataset/steam_games_genres.parquet", columns=['item_id', genero]).to_pandas()
    user_items_list_df = pq.read_table("dataset/user_items_list.parquet", columns=['item_id', 'steam_id', 'playtime_forever']).to_pandas()
    steam_games_df = pq.read_table("dataset/steam_games.parquet", columns=['item_id', 'release_year']).to_pandas()
    user_items_df = pq.read_table("dataset/user_items.parquet", columns=['steam_id', 'user_id']).to_pandas()

    # Filtrar los juegos que pertenecen al género especificado
    games_in_genre = steam_games_genre_df[steam_games_genre_df[genero] == 1]

    if games_in_genre.empty:
        return {"error": "Género no encontrado"}

    # Obtener una lista de los item_id de los juegos en el género
    item_ids_in_genre = games_in_genre['item_id'].tolist()

    # Filtrar los registros de user_items_list para obtener las horas jugadas de los juegos en el género
    genre_playtime = user_items_list_df[user_items_list_df['item_id'].isin(item_ids_in_genre)]

    if genre_playtime.empty:
        return {"error": "No hay datos de horas jugadas para este género"}

    # Agregar eficientemente las horas jugadas por usuario y año
    user_hours = (genre_playtime
                  .merge(steam_games_df, on='item_id')
                  .merge(user_items_df, left_on='steam_id', right_on='steam_id')
                  .groupby(['user_id', 'release_year'])
                  .agg(Horas=('playtime_forever', 'sum'))
                  .reset_index()
                  )

    # Encontrar el usuario con más horas jugadas para el género
    user_with_most_playtime = user_hours.groupby('user_id')['Horas'].sum().idxmax()

    # Filtrar las horas jugadas por usuario
    user_hours = user_hours[user_hours['user_id'] == user_with_most_playtime]

    # Crear el resultado final
    result = {
        "Usuario con más horas jugadas para género " + genero: user_with_most_playtime,
        "Horas jugadas": user_hours[['release_year', 'Horas']].to_dict(orient='records')
    }

    return result

In [2]:
f3 = UserForGenre('Action')
f3

{'Usuario con más horas jugadas para género Action': 'Sp3ctre',
 'Horas jugadas': [{'release_year': 1982, 'Horas': 1657},
  {'release_year': 1993, 'Horas': 0},
  {'release_year': 1995, 'Horas': 217},
  {'release_year': 1996, 'Horas': 0},
  {'release_year': 1997, 'Horas': 242},
  {'release_year': 1998, 'Horas': 0},
  {'release_year': 1999, 'Horas': 44},
  {'release_year': 2000, 'Horas': 70644},
  {'release_year': 2001, 'Horas': 250},
  {'release_year': 2002, 'Horas': 238},
  {'release_year': 2003, 'Horas': 7744},
  {'release_year': 2004, 'Horas': 127411},
  {'release_year': 2005, 'Horas': 21339},
  {'release_year': 2006, 'Horas': 90253},
  {'release_year': 2007, 'Horas': 112794},
  {'release_year': 2008, 'Horas': 8533},
  {'release_year': 2009, 'Horas': 108771},
  {'release_year': 2010, 'Horas': 82766},
  {'release_year': 2011, 'Horas': 155459},
  {'release_year': 2012, 'Horas': 385464},
  {'release_year': 2013, 'Horas': 122158},
  {'release_year': 2014, 'Horas': 132864},
  {'release_ye

# Cuarta funcion

## Función para obtener el top 3 de desarrolladoras por año

In [8]:
import pyarrow.parquet as pq
import pandas as pd

def UsersBestDeveloper(año: int):
    # Cargar los archivos Parquet y seleccionar las columnas necesarias
    steam_games_df = pq.read_table("dataset/steam_games.parquet", columns=['item_id', 'release_year', 'developer']).to_pandas()
    user_reviews_df = pq.read_table("dataset/user_reviews.parquet", columns=['item_id', 'recommend', 'sentiment_analysis']).to_pandas()

    # Paso 1: Filtrar el DataFrame de steam_games para obtener los juegos del año dado
    games_of_year = steam_games_df[steam_games_df['release_year'] == año]

    if games_of_year.empty:
        return {"error": "No hay juegos para el año especificado"}

    # Paso 2: Filtrar el DataFrame de user_reviews para obtener las reviews recomendadas y positivas
    positive_reviews = user_reviews_df[(user_reviews_df['recommend'] == 1) & (user_reviews_df['sentiment_analysis'] == 2)]

    # Paso 3: Contar las reviews recomendadas y positivas por juego
    recommendations_per_game = positive_reviews['item_id'].value_counts().reset_index()
    recommendations_per_game.columns = ['item_id', 'recommendation_per_game']

    # Paso 4: Fusionar los DataFrames de juegos y recomendaciones
    merged_df = games_of_year.merge(recommendations_per_game, on='item_id', how='inner')

    # Paso 5: Calcular la suma de recomendaciones por desarrollador
    developer_recommendations = merged_df.groupby('developer')['recommendation_per_game'].sum().reset_index()

    # Paso 6: Ordenar los desarrolladores por recomendaciones en orden descendente
    top_developers = developer_recommendations.nlargest(3, 'recommendation_per_game')

    # Paso 7: Formatear el resultado en el formato deseado
    result = [{"Puesto " + str(i + 1): developer} for i, developer in enumerate(top_developers['developer'])]

    return result

In [25]:
f4 = UsersBestDeveloper(2017)
f4

[{'Puesto 1': 'Smartly Dressed Games'},
 {'Puesto 2': 'Freejam'},
 {'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}]

# Quinta funcion

## Función para obtener la percepcion segun sentiment_analysis de una desarrolladora en cuanto a positivos o negativos

In [1]:
import pandas as pd
import pyarrow.parquet as pq

def DeveloperSentimentAnalysis(desarrolladora: str):
    # Cargar los archivos Parquet y seleccionar las columnas necesarias
    steam_games_df = pq.read_table("dataset/steam_games.parquet", columns=['item_id', 'developer']).to_pandas()
    user_reviews_df = pq.read_table("dataset/user_reviews.parquet", columns=['item_id', 'sentiment_analysis']).to_pandas()

    # Paso 1: Filtrar el DataFrame de steam_games para obtener los juegos del desarrollador
    games_of_developer = steam_games_df[steam_games_df['developer'] == desarrolladora]

    if games_of_developer.empty:
        return {"error": "Desarrolladora no encontrada"}

    # Paso 2: Filtrar el DataFrame de user_reviews para obtener las reviews relacionadas a estos juegos
    reviews_of_developer = user_reviews_df[user_reviews_df['item_id'].isin(games_of_developer['item_id'])]

    if reviews_of_developer.empty:
        return {"error": "No hay reseñas para juegos de esta desarrolladora"}

    # Paso 3: Filtrar las reseñas con sentiment_analysis igual a 2 (positivo) y 0 (negativo)
    positive_reviews = reviews_of_developer[reviews_of_developer['sentiment_analysis'] == 2]
    neutral_reviews = reviews_of_developer[reviews_of_developer['sentiment_analysis'] == 1]
    negative_reviews = reviews_of_developer[reviews_of_developer['sentiment_analysis'] == 0]

    # Paso 4: Contar la cantidad de reseñas positivas y negativas
    positive_count = len(positive_reviews)
    neutral_count = len(neutral_reviews)
    negative_count = len(negative_reviews)

    # Paso 5: Crear el diccionario de resultado
    result = {desarrolladora: {"Negative": negative_count, "Neutral": neutral_count, "Positive": positive_count}}

    return result

In [2]:
f5 = DeveloperSentimentAnalysis('Valve')
f5

{'Valve': {'Negative': 1116, 'Neutral': 2357, 'Positive': 6106}}